In [5]:
import re
import requests
from io import StringIO
from datetime import datetime, timedelta
import pandas as pd
import urllib
import gzip

In [6]:
# credit:
# https://stackoverflow.com/questions/44225771/scraping-historical-data-from-yahoo-finance-with-python?rq=1

In [7]:
class YFH: #YFH = YahooFinanceHistory
    timeout = 2
    crumb_link = 'https://finance.yahoo.com/quote/{0}/history?p={0}'
    crumble_regex = r'CrumbStore":{"crumb":"(.*?)"}'
    quote_link = 'https://query1.finance.yahoo.com/v7/finance/download/{quote}?period1={dfrom}&period2={dto}&interval=1d&events=history&crumb={crumb}'
    
    def __init__(self, symbol, days_back=7):
        self.symbol = symbol
        self.session = requests.Session()
        self.dt = timedelta(days=days_back)
        
    def get_crumb(self):
        response = self.session.get(self.crumb_link.format(self.symbol), timeout=self.timeout)
        response.raise_for_status()
        match = re.search(self.crumble_regex, response.text)
        if not match:
            raise ValueError('Could not get crumb from Yahoo Finance')
        else: 
            self.crumb = match.group(1)
        
    def get_quote(self):
        if not hasattr(self, 'crumb') or len(self.session.cookies) == 0:
            self.get_crumb()
        now = datetime.utcnow()
        dateto = int(now.timestamp())
        datefrom = int((now - self.dt).timestamp())
        url = self.quote_link.format(quote=self.symbol, dfrom=datefrom, dto=dateto, crumb=self.crumb)
        response = self.session.get(url)
        response.raise_for_status()
        return pd.read_csv(StringIO(response.text), parse_dates=['Date'])
        

In [8]:
# df = YFH('AAPL', days_back=30).get_quote()
# df['Symbol'] = 'AAPL'
# cols = df.columns.tolist()
# cols = cols[-1:] + cols[:-1]
# df = df[cols]
# df

In [9]:
import csv
nasdaq_url = 'https://www.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=nasdaq&render=download'
nyse_url = 'https://www.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=nyse&render=download'
amex_url = 'https://www.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=amex&render=download'

nasdaq_csv = urllib.request.urlopen(nasdaq_url)
nyse_csv = urllib.request.urlopen(nyse_url)
amex_csv = urllib.request.urlopen(amex_url)

nasdaq = pd.read_csv(nasdaq_csv)
nyse = pd.read_csv(nyse_csv)
amex = pd.read_csv(amex_csv)

nasdaq['Exchange'] = 'nasdaq'
nyse['Exchange'] = 'nyse'
amex['Exchange'] = 'amex'

symbols = pd.DataFrame(columns=nyse.columns.tolist())

In [10]:
symbols = nasdaq
# cols = symbols.columns.tolist()
# cols = cols[-1:] + cols[:-1]
# symbols = symbols[cols]

symbols.append(nyse)
symbols.append(amex)

exchanges = list(symbols['Exchange'])
symbols = list(symbols['Symbol'])

# print(symbols[0:9], exchanges[0:9])

In [33]:
df = pd.DataFrame()

for i in symbols:
    print(i)
    
    try:
        df_i = YFH(i, days_back=365).get_quote()
        df_i['Symbol'] = i
        df = df.append(df_i)
        print(i)
    except: ## skipping when there is a cookie / crumb issue with yahoo finance...
        pass

cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

YI
YI
PIH
PIH
PIHPP
PIHPP
TURN
TURN
FLWS
FLWS
FCCY
FCCY
SRCE
SRCE
VNET
VNET
TWOU


In [34]:
df.to_csv("/home/david/stockPortfolioGen/data/stocks.csv.gz", compression='gzip')

In [35]:
df = pd.read_csv('/home/david/stockPortfolioGen/data/stocks.csv.gz', compression='gzip')
# df

,Unnamed: 0,Symbol,Date,Open,High,Low,Close,Adj Close,Volume
0,0,YI,2018-09-12,16.75,16.83,11.58,13.80,13.80,1302200.0
1,1,YI,2018-09-13,13.90,14.22,13.00,14.00,14.00,685200.0
2,2,YI,2018-09-14,14.60,14.73,13.05,13.99,13.99,773400.0
3,3,YI,2018-09-17,13.61,13.92,13.25,13.72,13.72,198000.0
4,4,YI,2018-09-18,13.85,13.85,12.70,13.06,13.06,259200.0
5,5,YI,2018-09-19,13.26,13.82,12.71,13.82,13.82,96000.0
6,6,YI,2018-09-20,13.82,13.82,13.12,13.51,13.51,56000.0
7,7,YI,2018-09-21,13.50,15.50,13.50,15.44,15.44,297000.0
8,8,YI,2018-09-24,15.12,15.12,14.31,14.81,14.81,76000.0
9,9,YI,2018-09-25,14.80,14.80,13.66,14.07,14.07,91500.0
